In [1]:
import pickle

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import gym

%matplotlib inline

In [2]:
hidden_layer_num = 3
hidden_layer_size = 49

# Visualize performance

In [3]:
# TODO: this should be refactored and DRY from tuning-hyperparameters-and-visualization/train_humanoid.py

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


with open('./train_test_data/humanoid_train_test.pkl', 'rb') as inf:
    X_tv, y_tv, X_test, y_test = pickle.load(inf)

tf.logging.info('{0}, {1}, {2}, {3}'.format(
    X_tv.shape, X_test.shape, y_tv.shape, y_test.shape
))

x_plh = tf.placeholder(tf.float32, shape=[None, X_tv.shape[1]])
y_plh = tf.placeholder(tf.float32, shape=[None, y_tv.shape[1]])

with tf.name_scope('fc1'):
    Wh_var = weight_variable([x_plh.shape.dims[1].value, hidden_layer_size])
    bh_var = bias_variable([hidden_layer_size])
    hh = tf.nn.sigmoid(tf.matmul(x_plh, Wh_var) + bh_var)

for i in range(hidden_layer_num - 1):
    with tf.name_scope('fc{0}'.format(i + 2)):
        Wh_var = weight_variable([hidden_layer_size, hidden_layer_size])
        bh_var = bias_variable([hidden_layer_size])
        hh = tf.nn.sigmoid(tf.matmul(hh, Wh_var) + bh_var)

with tf.name_scope('out'):
    W_var = weight_variable([hidden_layer_size, y_plh.shape.dims[1].value])
    b_var = bias_variable([y_plh.shape.dims[1].value])
    y_pred = tf.matmul(hh, W_var) + b_var

INFO:tensorflow:(160080, 376), (40020, 376), (160080, 17), (40020, 17)


In [4]:
saver = tf.train.Saver()

sess = tf.InteractiveSession()
saver.restore(sess, "./49/3-49")

INFO:tensorflow:Restoring parameters from ./49/3-49


In [5]:
def pred_action(obs):
    return y_pred.eval(feed_dict={x_plh: obs.reshape(1, -1)})

env = gym.make('Humanoid-v1')

totalr_list = []
max_timesteps = 600
for _ in range(3):
    print(_, end=',')
    totalr = 0
    obs = env.reset()
    for k in range(max_timesteps):
        action = pred_action(obs[None,:])
        obs, r, done, _ = env.step(action)
        totalr += r
        env.render()
    env.render(close=True)
    totalr_list.append(totalr)
print()
print(totalr_list)
print(np.mean(totalr_list))
print(np.std(totalr_list))

0,1,2,
[6018.7419806805347, 6092.0598468783646, 6108.5401256820496]
6073.11398441
39.0310644643


#### Expert performance

Collected by running `python run_expert.py experts/Humanoid-v1.pkl Humanoid-v1 --max_timesteps 600 --num_rollouts 3`

Expert's return at 60 timestep is around 330.
Expert's return at 100 timestep is around 644.
Expert's return at 600 timestep is around 6018

**NOTE (Deprecated)**: the performance is only comparable at small number of timesteps. Otherwise, likely the error grow quickly and the performance deteriorate fast.

To improve:

1. The NN is too simplistic
1. Not enough training data
1. Not training for long enough (Training is too slow currently, GPU not used effectively)

**Updated Note**: tuning the network size really matters, currently it's 3 hidden layers and 49 neurons per hidden layer.